<a href="https://colab.research.google.com/github/Moe-phantom/NEURO_SYMBOLIC_AI/blob/main/NEURO_SYMBOLIC_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import ddgs # The new package name
    import datasets
    import transformers
    import spacy
except ImportError:
    print("Installing required libraries...")
    install("datasets")
    install("transformers")
    install("torch")
    install("scikit-learn")
    install("spacy")
    install("ddgs") # CHANGED from duckduckgo-search
    install("accelerate")
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])

Installing required libraries...


In [2]:
import torch
import torch.nn as nn
import json
import os
import time
import random
import warnings
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from ddgs import DDGS
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score



In [16]:
# CLEANUP
warnings.filterwarnings("ignore")

# CONFIGURATION
MODEL_NAME = "distilbert-base-uncased"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16  # Smaller batch size due to attention memory
EPOCHS = 4
MAX_LEN = 256
CACHE_FILE = "evidence_cache_v3.json"  # <--- Defined HERE so the class can see it

In [17]:
from datasets import load_dataset

def prepare_data(split="train", limit=None):
    """
    Loads LIAR data but STRIPS the 'justification' (the cheat).
    Instead, it fetches REAL web evidence for the claim.
    """
    print(f"Loading {split} dataset...")
    ds = load_dataset("chengxuphd/liar2", split=split)

    data = []
    count = 0

    print(f"Fetching evidence for {split} set (this may take time)...")
    for x in tqdm(ds):
        if limit and count >= limit:
          break

        stmt = str(x.get("statement", ""))

        # LABEL CLEANING:
        # 0=barely, 1=false, 2=half(AMBIGUOUS), 3=mostly, 4=pants, 5=true
        label_map = {5: 1.0, 3: 1.0, 1: 0.0, 0: 0.0, 4: 0.0} # Skip 2 (Half-True)
        raw_label = x["label"]

        if raw_label not in label_map: continue # Skip ambiguous data

        # CRITICAL STEP: Fetch evidence like a real user would
        # For training speed, we might rely on the cache heavily
        evidence = searcher.search(stmt)

        data.append({
            "claim": stmt,
            "evidence": evidence,
            "label": label_map[raw_label]
        })
        count += 1

    return data


In [18]:

# 2. WEB SEARCH
class SafeWebSearch:
    def __init__(self, cache_file=CACHE_FILE):
        self.cache_file = cache_file

        # Load existing cache if available
        if os.path.exists(cache_file):
            with open(cache_file, 'r') as f:
                self.cache = json.load(f)
        else:
            self.cache = {}

        self.ddgs = DDGS()

    def search(self, query):
        # 1. Check Cache
        if query in self.cache:
            return self.cache[query]

        # 2. Live Search
        try:
            results = list(self.ddgs.text(query, max_results=3))

            if not results:
                evidence = "No evidence found."
            else:
                evidence = " ".join([r.get("body", "") for r in results])

            # Update Cache
            self.cache[query] = evidence
            self._save_cache()
            return evidence

        except Exception as e:
            print(f" [!] Search Warning: {e}")
            time.sleep(1) # Short cool-down
            return "Evidence unavailable."

    def _save_cache(self):
        with open(self.cache_file, 'w') as f:
            json.dump(self.cache, f)

# Initialize Searcher
searcher = SafeWebSearch()
print(f"Searcher ready. Loaded {len(searcher.cache)} items from memory.")

Searcher ready. Loaded 159 items from memory.


In [19]:
BACKUP_CACHE = "cache_backup.json"       # The one you just uploaded

def merge_caches():
    print("Beginning Safe Merge...")

    # 1. Load the Backup (Your saved progress)
    if not os.path.exists(BACKUP_CACHE):
        print(f"❌ Error: Could not find '{BACKUP_CACHE}'. Did you upload it?")
        return

    with open(BACKUP_CACHE, 'r') as f:
        backup_data = json.load(f)
    print(f"✅ Loaded Backup: {len(backup_data)} items found.")

    # 2. Load the Current Session Cache (If it exists)
    if os.path.exists(CURRENT_CACHE):
        with open(CURRENT_CACHE, 'r') as f:
            current_data = json.load(f)
        print(f"✅ Loaded Current Session: {len(current_data)} items found.")
    else:
        current_data = {}
        print("ℹ️ Current session is empty. Creating new.")

    # 3. MERGE (The backup fills in the blanks)
    # We update current_data with backup_data.
    # This keeps new stuff, but fills gaps with old stuff.
    initial_count = len(current_data)

    for question, answer in backup_data.items():
        if question not in current_data:
            current_data[question] = answer

    final_count = len(current_data)
    added = final_count - initial_count

    # 4. Save the Combined Brain
    with open(CURRENT_CACHE, 'w') as f:
        json.dump(current_data, f)

    print("="*40)
    print(f"🎉 MERGE COMPLETE!")
    print(f"   - Items added from backup: {added}")
    print(f"   - Total knowledge available: {final_count}")
    print("="*40)

# Run the merge
merge_caches()

Beginning Safe Merge...
✅ Loaded Backup: 5398 items found.
✅ Loaded Current Session: 159 items found.
🎉 MERGE COMPLETE!
   - Items added from backup: 5239
   - Total knowledge available: 5398


In [20]:
# 2. DATASET (USING REAL SEARCH)
from datasets import load_dataset

def prepare_data(split="train", limit=None):
    """
    Loads LIAR data but STRIPS the 'justification' (the cheat).
    Instead, it fetches REAL web evidence for the claim.
    """
    print(f"Loading {split} dataset...")
    ds = load_dataset("chengxuphd/liar2", split=split)

    data = []
    count = 0

    print(f"Fetching evidence for {split} set (this may take time)...")
    for x in tqdm(ds):
        if limit and count >= limit: break

        stmt = str(x.get("statement", ""))

        # LABEL CLEANING:
        # 0=barely, 1=false, 2=half(AMBIGUOUS), 3=mostly, 4=pants, 5=true
        label_map = {5: 1.0, 3: 1.0, 1: 0.0, 0: 0.0, 4: 0.0} # Skip 2 (Half-True)
        raw_label = x["label"]

        if raw_label not in label_map: continue # Skip ambiguous data

        # CRITICAL STEP: Fetch evidence like a real user would
        # For training speed, we might rely on the cache heavily
        evidence = searcher.search(stmt)

        data.append({
            "claim": stmt,
            "evidence": evidence,
            "label": label_map[raw_label]
        })
        count += 1

    return data

In [21]:
# LIMIT DATA FOR DEMO (Remove 'limit=500' for full training)
train_raw = prepare_data("train", limit=5000)
val_raw = prepare_data("validation", limit=400)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class EvidenceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # encode Claim and Evidence SEPARATELY
        # This allows the Attention Mechanism to compare them
        claim_enc = tokenizer(item["claim"], truncation=True, max_length=64, padding="max_length", return_tensors="pt")
        evid_enc = tokenizer(item["evidence"], truncation=True, max_length=256, padding="max_length", return_tensors="pt")

        return {
            "claim_ids": claim_enc["input_ids"].squeeze(0),
            "claim_mask": claim_enc["attention_mask"].squeeze(0),
            "evid_ids": evid_enc["input_ids"].squeeze(0),
            "evid_mask": evid_enc["attention_mask"].squeeze(0),
            "label": torch.tensor(item["label"], dtype=torch.float)
        }

train_loader = DataLoader(EvidenceDataset(train_raw), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(EvidenceDataset(val_raw), batch_size=BATCH_SIZE)

Loading train dataset...
Fetching evidence for train set (this may take time)...


  0%|          | 0/18369 [00:00<?, ?it/s]

Loading validation dataset...
Fetching evidence for validation set (this may take time)...


  0%|          | 0/2297 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [23]:
# 3. ARCHITECTURE: EVIDENCE ATTENTION NETWORK
# ============================================================================
class EvidenceAttentionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = AutoModel.from_pretrained(MODEL_NAME)
        hidden_dim = self.bert.config.hidden_size # 768

        # CROSS-ATTENTION LAYER
        # Query = Claim, Key/Value = Web Evidence
        self.attn = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=4, batch_first=True)

        # Classification Head
        # Input: [Claim_Vector + Evidence_Context_Vector]
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)
        )
    def forward(self, c_ids, c_mask, e_ids, e_mask):
        # 1. Encode Claim
        # shape: (batch, seq_len, 768)
        claim_out = self.bert(c_ids, c_mask).last_hidden_state
        # Get CLS token for claim summary
        claim_cls = claim_out[:, 0, :]

        # 2. Encode Evidence
        evid_out = self.bert(e_ids, e_mask).last_hidden_state

        # 3. ATTENTION MECHANISM
        # "What parts of the Evidence (Key) match the Claim (Query)?"
        # Q=Claim(Sequence), K=Evidence(Sequence), V=Evidence(Sequence)
        # We use the full sequence for attention, not just CLS
        attn_output, _ = self.attn(claim_out, evid_out, evid_out, key_padding_mask=~e_mask.bool())

        # Pool the attention output (Summarize the "verified" parts)
        # We take the CLS position of the attention output which corresponds to Claim's CLS attending to Evidence
        context_vec = attn_output[:, 0, :]

        # 4. Fusion
        # Combine the original Claim with the Found Context
        fused = torch.cat([claim_cls, context_vec], dim=1)

        logits = self.classifier(fused)
        return logits

# Initialize
model = EvidenceAttentionNet().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
pos_weight = torch.tensor([2.0]).to(DEVICE)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [24]:
def train():
    print("\nStarting Training with Weighted Loss & Validation...")

    for epoch in range(EPOCHS):
        # --- PHASE 1: TRAINING ---
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
            c_ids = batch["claim_ids"].to(DEVICE)
            c_mask = batch["claim_mask"].to(DEVICE)
            e_ids = batch["evid_ids"].to(DEVICE)
            e_mask = batch["evid_mask"].to(DEVICE)
            labels = batch["label"].to(DEVICE).unsqueeze(1)

            optimizer.zero_grad()
            logits = model(c_ids, c_mask, e_ids, e_mask)
            loss = criterion(logits, labels) # Uses the pos_weight=2.0
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # --- PHASE 2: VALIDATION ---
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
                c_ids = batch["claim_ids"].to(DEVICE)
                c_mask = batch["claim_mask"].to(DEVICE)
                e_ids = batch["evid_ids"].to(DEVICE)
                e_mask = batch["evid_mask"].to(DEVICE)
                labels = batch["label"].to(DEVICE).unsqueeze(1)

                logits = model(c_ids, c_mask, e_ids, e_mask)
                preds = (torch.sigmoid(logits) > 0.5).float()

                correct += (preds == labels).sum().item()
                total += labels.size(0)

        print(f"Epoch {epoch+1} Result:")
        print(f"  - Avg Loss: {total_loss/len(train_loader):.4f}")
        print(f"  - Accuracy: {(correct/total)*100:.2f}%")
        print("-" * 40)

In [25]:
def test_live(claim):
    model.eval()
    print(f"\n[TEST] Claim: {claim}")

    # 1. Search
    evidence = searcher.search(claim)
    print(f"  > Evidence: {evidence[:100]}...")

    # 2. Tokenize Separately (Dual-Encoder)
    c_enc = tokenizer(claim, return_tensors="pt", truncation=True, max_length=64).to(DEVICE)
    e_enc = tokenizer(evidence, return_tensors="pt", truncation=True, max_length=256).to(DEVICE)

    # 3. Predict
    with torch.no_grad():
        logits = model(c_enc.input_ids, c_enc.attention_mask, e_enc.input_ids, e_enc.attention_mask)
        prob = torch.sigmoid(logits).item()

    verdict = "REAL" if prob > 0.5 else "FAKE"
    print(f"  > Verdict: {verdict} ({prob:.1%})")

# Run it
if __name__ == "__main__":
    train()
    test_live("The earth is flat.")
    test_live("Python is a programming language.")


Starting Training with Weighted Loss & Validation...


Epoch 1 [Train]:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch 1 [Val]:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1 Result:
  - Avg Loss: 0.8287
  - Accuracy: 64.00%
----------------------------------------


Epoch 2 [Train]:   0%|          | 0/313 [00:00<?, ?it/s]

...

Epoch 3 [Val]:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 3 Result:
  - Avg Loss: 0.5876
  - Accuracy: 65.25%
----------------------------------------


Epoch 4 [Train]:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch 4 [Val]:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 4 Result:
  - Avg Loss: 0.4103
  - Accuracy: 70.00%
----------------------------------------

[TEST] Claim: The earth is flat.
  > Evidence: Flat Earth is an archaic and scientifically disproven conception of the Earth's shape as a plane or ...
  > Verdict: FAKE (29.5%)

[TEST] Claim: Python is a programming language.
  > Evidence: Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms...
  > Verdict: REAL (52.5%)


In [27]:

# Test on modern news (Model has never seen these!)
test_live("The sun rises in the west.")
test_live("LeBron James is better than Michael Jordan")
test_live("Hitler and germany won the 2nd world war")


[TEST] Claim: The sun rises in the west.
  > Evidence: Sun path on a polar graph for any location at the latitude of Rotterdam. This solargraph exposed ove...
  > Verdict: FAKE (0.1%)

[TEST] Claim: LeBron James is better than Michael Jordan
  > Evidence: Nov 26, 2024 · Michael Jordan may be 6-0 in the Finals, but LeBron has a far better record than MJ a...
  > Verdict: FAKE (1.7%)

[TEST] Claim: Hitler and germany won the 2nd world war
  > Evidence: World War II is generally considered to have begun on 1 September 1939, when Nazi Germany , under Ad...
  > Verdict: FAKE (0.1%)


In [31]:
test_live("Python is a programming language")


[TEST] Claim: Python is a programming language
  > Evidence: Python is a high-level, general-purpose programming language . Its design philosophy emphasizes code...
  > Verdict: REAL (71.3%)


In [32]:
test_live("C++ is a programming language")


[TEST] Claim: C++ is a programming language
  > Evidence: Ciao is a general-purpose programming language which supports logic, constraint, functional, higher-...
  > Verdict: FAKE (31.0%)


In [33]:
import torch
from google.colab import files

# Save to disk
torch.save(model.state_dict(), "final_model_70acc.pth")
print("✅ Model saved locally.")

# Download to your computer
files.download("final_model_70acc.pth")

✅ Model saved locally.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
test_live("c is programming language used mostly for hardware ")


[TEST] Claim: c is programming language used mostly for hardware 
  > Evidence: C is an imperative procedural language , supporting structured programming , lexical variable scope,...
  > Verdict: REAL (96.4%)
